# ECE 258 HW1

## Pin Tian

### PID: A53219987

In [1]:
import numpy as np
import urllib
import scipy.optimize
import random

def parseData(fname):
    for l in urllib.request.urlopen(fname):
        yield eval(l)

print ("Reading data...")
data = list(parseData("http://jmcauley.ucsd.edu/cse255/data/beer/beer_50000.json"))
print ("done")

Reading data...
done


## regression

## Problem 1

(1) The table of the number of reviews for each style of beer( in alphabet order) is shown below:

In [2]:
from collections import Counter, defaultdict

counter = Counter([d['beer/style'] for d in data])
styles = []
for key in sorted(counter.keys()):
    styles.append([key, counter[key]])
for style, total in styles:
    print ('{:50}\t{}'.format(style, total))

Altbier                                           	165
American Adjunct Lager                            	242
American Amber / Red Ale                          	665
American Amber / Red Lager                        	42
American Barleywine                               	825
American Black Ale                                	138
American Blonde Ale                               	357
American Brown Ale                                	314
American Dark Wheat Ale                           	14
American Double / Imperial IPA                    	3886
American Double / Imperial Pilsner                	14
American Double / Imperial Stout                  	5964
American IPA                                      	4113
American Malt Liquor                              	90
American Pale Ale (APA)                           	2288
American Pale Lager                               	123
American Pale Wheat Ale                           	154
American Porter                                   	2230
American 

(2) The table of the average value of 'review/taste' for reviews from each style is shown below:

In [3]:
average = defaultdict(int)
for d in data:
    style, taste = d['beer/style'], d['review/taste']
    average[style] += taste / counter[style]
for style, taste in sorted(average.items()):
    print ('{:50}\t{}'.format(style, taste))

Altbier                                           	3.4030303030302997
American Adjunct Lager                            	2.94834710743802
American Amber / Red Ale                          	3.5135338345864495
American Amber / Red Lager                        	3.6904761904761916
American Barleywine                               	4.0642424242424156
American Black Ale                                	3.873188405797102
American Blonde Ale                               	3.2549019607843044
American Brown Ale                                	3.7436305732484128
American Dark Wheat Ale                           	3.6785714285714284
American Double / Imperial IPA                    	4.033324755532863
American Double / Imperial Pilsner                	3.8214285714285707
American Double / Imperial Stout                  	4.479963112005273
American IPA                                      	4.000850960369668
American Malt Liquor                              	2.255555555555554
American Pale Ale (APA)    

## Problem 2

In [4]:
def feature(datum):
    feat = [1]
    if datum['beer/style'] == "American IPA":
        feat.append(1)
    else:
        feat.append(0)
    return feat

X = [feature(d) for d in data]
tastes = [d['review/taste'] for d in data]
theta,residuals,rank,s = np.linalg.lstsq(X, tastes)
print ('theta_0 = %f' % (theta[0]))
print ('theta_1 = %f' % (theta[1]))

theta_0 = 3.915205
theta_1 = 0.085646


C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:11: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  # This is added back by InteractiveShellApp.init_path()


The bias such as the average 'review/taste' value for beer that is not American IPA is shown by $\theta_0$. 

$\theta_1$ shows how the 'review/taste' value of 'American IPA' differs from the average of other styles of beer. Because $\theta_1$ is bigger than 0, it indicates that the 'review/taste' value of American IPA is $0.085646$ higher than the average of other styles of beer.

## Problem 3

In [5]:
numrows = len(X)
X_train, X_test = X[:numrows // 2], X[numrows // 2:]
tastes_train, tastes_test = tastes[:numrows // 2], tastes[numrows // 2:]
theta,residuals,rank,s = np.linalg.lstsq(X_train, tastes_train)
print ('theta_0 = %f' % (theta[0]))
print ('theta_1 = %f' % (theta[1]))

tastes_train_predict = np.dot(X_train, theta)
tastes_test_predict = np.dot(X_test, theta)

MSE_train = np.mean((tastes_train-tastes_train_predict)**2)
MSE_test = np.mean((tastes_test-tastes_test_predict)**2)

print ('MSE_train = %f' % (MSE_train))
print ('MSE_test = %f' % (MSE_test))

theta_0 = 3.904356
theta_1 = 0.056060
MSE_train = 0.558107
MSE_test = 0.468410


C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:4: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  after removing the cwd from sys.path.


## Problem 4

In [6]:
styles_50 = sorted([style for style in counter 
                       if counter[style] >= 50])
index_styles = {style: i + 1 for i, style in enumerate(styles_50)}

def feature(datum):
    feat = [1] + [0] * len(styles_50)
    style = datum['beer/style']
    if style in index_styles:
        feat[index_styles[style]] = 1
    return feat

X = [feature(d) for d in data]
y = [d['review/taste'] for d in data]
X_train, X_test = X[:numrows // 2], X[numrows // 2:]
y_train, y_test = y[:numrows // 2], y[numrows // 2:]
theta,residuals,rank,s = np.linalg.lstsq(X_train, y_train)
print('theta =\n %s' % theta)

y_train_predict = np.dot(X_train, theta)
y_test_predict = np.dot(X_test, theta)

MSE_train = np.mean((y_train-y_train_predict)**2)
MSE_test = np.mean((y_test-y_test_predict)**2)

print ('MSE_train = %f' % (MSE_train))
print ('MSE_test = %f' % (MSE_test))

C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  app.launch_new_instance()


theta =
 [ 3.60681818 -0.18884943 -0.73802386 -0.11634199  0.45638407  0.19621212
 -0.45410882  0.26761104  0.33596789  0.8416167   0.35359848 -1.00681818
  0.02739474 -0.58598485 -0.17824675  0.3305986   0.20312334 -0.1798951
  0.58195733  0.65508658 -0.27061129  0.3449362   0.16385851  0.12651515
  0.46842454  0.33580477 -0.74318182 -0.65227273 -0.2937747  -0.41285266
  0.13786267  0.25681818  0.76540404 -0.02450111  0.03420746 -0.13106061
 -0.31931818  0.09815076 -0.10223103  0.12193999  0.11433566 -0.92019846
 -0.83277972  0.14466111 -0.35681818  0.11320382 -0.62765152 -0.0058248
 -0.38622995  0.22651515 -0.63806818  0.10049889 -1.23390152 -0.10681818
  0.71136364 -0.50681818 -0.20681818 -0.22045455  0.1527972   0.60472028
  0.26709486  0.39318182  0.44318182  0.69564175  0.3763751   0.20984848
  0.12395105  0.48544372  0.26818182  0.38356643 -0.4664673   0.05895722
  0.01818182  0.01699134 -0.09003966]
MSE_train = 0.367840
MSE_test = 0.433670


## Classification

## Problem 5

In [7]:
from sklearn import svm
def feature(datum):
    return [datum['beer/ABV'], datum['review/taste']]

X = [feature(d) for d in data]
y = ["American IPA" in b['beer/style'] for b in data]

X_train, X_test = X[:numrows // 2], X[numrows // 2:]
y_train, y_test = y[:numrows // 2], y[numrows // 2:]

clf = svm.SVC(C=1000)
clf.fit(X_train, y_train)

C:\ProgramData\Miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [8]:
train_predict = clf.predict(X_train)
test_predict = clf.predict(X_test)
accuracy_train = np.mean(train_predict == y_train)
accuracy_test = np.mean(test_predict == y_test)
print('The accuracy on the train data is %.2f%%' % (accuracy_train * 100))
print('The accuracy on the test data is %.2f%%' % (accuracy_test * 100))

The accuracy on the train data is 92.26%
The accuracy on the test data is 85.63%


## Problem 6

After observing the features from the text, I chose three more features 'review/appearance', 'review/palate' and the last one, the 'reviews/text' which including 'bitter' and 'amber' or 'orange' because the taste American IPA is bitterness and the appearance of American IPA is amber or orange. 

We can see that I got a better result of accuracy especially on the test set.

In addition, to expedite the training process, a normalization is done by linearly mapping training samples to [0, 1], i.e.
$$\widehat{x}=\frac{x - \min(x_{\text{train}}[n])}{\max(x_{\text{train}}[n] - \min(x_{\text{train}}[n]))}$$
Another reason for doing this normalization is that if it is not done, the SVM classifier won't converge when $C=100000$.

In [11]:
def feature(datum):
    feat = [datum['review/taste'],
            datum['beer/ABV'],
            1. if ('bitter' and ('amber' or 'orange')) in datum['review/text'] else 0.,
            datum['review/appearance'],
            datum['review/palate']]
    return feat

def normalization(features, training=False):
    features = np.asarray(features)
    if training:
        normalization.mins = np.min(features, axis=0)
        normalization.maxs = np.max(features, axis=0)
    return (features - normalization.mins) /  (normalization.maxs - normalization.mins)

X = [feature(d) for d in data]
y = ["American IPA" in b['beer/style'] for b in data]


X_train = normalization(X[:numrows // 2], training=True)
X_test = normalization(X[numrows // 2:], training=False)
# X_train, X_test = np.asarray(X[:numrows // 2]), np.asarray(X[numrows // 2:])
y_train, y_test = y[:numrows // 2], y[numrows // 2:]

clf = svm.SVC(C=1000)
clf.fit(X_train, y_train)

C:\ProgramData\Miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1000, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [12]:
train_predict = clf.predict(X_train)
test_predict = clf.predict(X_test)
accuracy_train = np.mean(train_predict == y_train)
accuracy_test = np.mean(test_predict == y_test)
print('The accuracy on the train data is %.2f%%' % (accuracy_train * 100))
print('The accuracy on the test data is %.2f%%' % (accuracy_test * 100))

The accuracy on the train data is 91.36%
The accuracy on the test data is 92.19%


## Problem 7

The regularization constant C in SVM is generally means that tolerance to the error samples. The larger C, the lower tolerance.

But much higher C may result overfitting and cause the compute speed lower. If the C is much smaller, the model won't work well as well. So to get a good performance, it is very important to choose an appropriate value of C.

According the results in the problem, increasing c does not effect significantly on our training set results. But it has more effects on my test set results. So the test features I choose may need change to other features.

In [13]:
for C in [0.1, 10, 1000, 100000]:
    clf = svm.SVC(C=C)
    clf.fit(X_train, y_train)
    train_predict = clf.predict(X_train)
    test_predict = clf.predict(X_test)    
    accuracy_train = np.mean(train_predict == y_train)
    accuracy_test = np.mean(test_predict == y_test)
    print ('C = %s' % str(C))
    print ('The accuracy on the train data is is %.2f%%' % (accuracy_train * 100))
    print ('The accuracy on the test data is %.2f%%' % (accuracy_test * 100))

C:\ProgramData\Miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C = 0.1
The accuracy on the train data is is 91.36%
The accuracy on the test data is 92.19%


C:\ProgramData\Miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C = 10
The accuracy on the train data is is 91.36%
The accuracy on the test data is 92.19%


C:\ProgramData\Miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C = 1000
The accuracy on the train data is is 91.36%
The accuracy on the test data is 92.19%


C:\ProgramData\Miniconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


C = 100000
The accuracy on the train data is is 91.56%
The accuracy on the test data is 91.47%


## Problem 8

In [14]:
import numpy
import urllib
import scipy.optimize
import random
from math import exp
from math import log
from scipy.special import expit

def parseData(fname):
    for l in urllib.urlopen(fname):
        yield eval(l)


def inner(x,y):
    return sum([x[i]*y[i] for i in range(len(x))])

def sigmoid(x):
    return 1.0 / (1 + exp(-x))

# NEGATIVE Log-likelihood
def f(theta, X, y, lam):
    loglikelihood = 0
    for i in range(len(X)):
        logit = inner(X[i], theta)
        loglikelihood -= log(1 + exp(-logit))
        if not y[i]:
            loglikelihood -= logit
    for k in range(len(theta)):
        loglikelihood -= lam * theta[k]*theta[k]
    print ("ll =", loglikelihood)
    return -loglikelihood

# NEGATIVE Derivative of log-likelihood
def fprime(theta, X, y, lam):
    dl = [0.0]*len(theta)
    for i in range(len(X)):
    # Fill in code for the derivative
        logit = sigmoid(inner(X[i], theta)) 
        dl -= (logit - y[i]) * X[i]
#         dl = list(map(lambda a, b: (a - (logit - y[i]) * b), zip(dl, X[i])))
    dl -= 2 * lam * theta
    pass
  # Negate the return value since we're doing gradient *ascent*
    return numpy.array([-x for x in dl])

# If we wanted to split with a validation set:
#X_valid = X[len(X)/2:3*len(X)/4]
#X_test = X[3*len(X)/4:]
# Use a library function to run gradient descent (or you can implement yourself!)
theta,l,info = scipy.optimize.fmin_l_bfgs_b(f, [0]*len(X[0]), fprime, args = (X_train, y_train, 1.0))
print ("Final log likelihood =", -l)

test_predict = numpy.dot(X_test, theta) > 0    
train_predict = numpy.dot(X_train, theta) > 0

accuracy_test = np.mean(test_predict == y_test)
accuracy_train = np.mean(train_predict == y_train)

print ('Accuracy = , %.2f%%' % (accuracy_test * 100))# Compute the accuracy)
print ('Accuracy = , %.2f%%' % (accuracy_train * 100))# Compute the accuracy)

ll = -17328.679513999832
ll = -9170.967174702992
ll = -8013.542622984981
ll = -7757.486442761354
ll = -7715.375144080942
ll = -7695.684038544232
ll = -7661.4969342425575
ll = -7632.061939036233
ll = -7615.227916256814
ll = -7577.9262281454685
ll = -7536.181336703033
ll = -7452.702321780031
ll = -7362.934068746758
ll = -7312.702711423136
ll = -7298.625222684866
ll = -7289.1850920370625
ll = -7266.50787459172
ll = -7225.066967286063
ll = -7168.263686090061
ll = -7137.501912725782
ll = -7133.470444456122
ll = -7131.807304637874
ll = -7131.662100045443
ll = -7131.616941570908
ll = -7131.5461901996105
ll = -7131.333102915313
ll = -7131.3302725683825
ll = -7131.325914682913
ll = -7131.3259025171665
Final log likelihood = -7131.3259025171665
Accuracy = , 92.19%
Accuracy = , 91.36%
